In [1]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoConfig, AutoModel 
import logging 


model_name = "FacebookAI/roberta-base"

model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)

#model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2', device = "mps") 
#tokenizer = model.tokenizer
from metrics import * 
start_token, sep_token, pad_token_id = tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token_id
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import os 

logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

/Users/croarkin.kyle/Desktop/dial2vec/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
def line_statistics(file_name):
    """
    统计文件行数
    """
    if file_name is None:
        return 0

    return 6948 


In [3]:
import torch.nn as nn 
import torch 

class PoolingAverage(nn.Module):
    def __init__(self, eps=1e-12):
        super(PoolingAverage, self).__init__()
        self.eps = eps

    def forward(self, hidden_states, attention_mask):
        mul_mask = lambda x, m: x * torch.unsqueeze(m, dim=-1)
        reduce_mean = lambda x, m: torch.sum(mul_mask(x, m), dim=1) / (torch.sum(m, dim=1, keepdims=True) + self.eps)

        avg_output = reduce_mean(hidden_states, attention_mask)
        return avg_output

    def equal_forward(self, hidden_states, attention_mask):
        mul_mask = hidden_states * attention_mask.unsqueeze(-1)
        avg_output = torch.sum(mul_mask, dim=1) / (torch.sum(attention_mask, dim=1, keepdim=True) + self.eps)
        return avg_output


In [4]:





class DialogueTransformer(nn.Module):
    def __init__(self):
        super(DialogueTransformer, self).__init__()
        self.bert = model 

        self.config = config

        self.dropout = nn.Dropout(self.config.hidden_dropout_prob)
        self.labels_data = None
        self.sample_nums = 10
        self.log_softmax = nn.LogSoftmax(dim=-1)
        self.avg = PoolingAverage(eps=1e-6)
        self.logger = logger  

    def forward(self, data, strategy='mean_by_role', output_attention=False):


        if len(data) == 7:
            input_ids, attention_mask, token_type_ids, role_ids, turn_ids, position_ids, labels = data
        else:
            input_ids, attention_mask, token_type_ids, role_ids, turn_ids, position_ids, labels, guids = data

        input_ids = input_ids.view(input_ids.size()[0] * input_ids.size()[1], input_ids.size()[-1])
        attention_mask = attention_mask.view(attention_mask.size()[0] * attention_mask.size()[1], attention_mask.size()[-1])
        token_type_ids = token_type_ids.view(token_type_ids.size()[0] * token_type_ids.size()[1], token_type_ids.size()[-1])
        role_ids = role_ids.view(role_ids.size()[0] * role_ids.size()[1], role_ids.size()[-1])
        turn_ids = turn_ids.view(turn_ids.size()[0] * turn_ids.size()[1], turn_ids.size()[-1])
        position_ids = position_ids.view(position_ids.size()[0] * position_ids.size()[1], position_ids.size()[-1])

        one_mask = torch.ones_like(role_ids)
        zero_mask = torch.zeros_like(role_ids)
        role_a_mask = torch.where(role_ids == 0, one_mask, zero_mask)
        role_b_mask = torch.where(role_ids == 1, one_mask, zero_mask)

        sep_token_id = tokenizer.sep_token_id 
        a_attention_mask = (attention_mask * role_a_mask)
        b_attention_mask = (attention_mask * role_b_mask)

        self_output, pooled_output = self.encoder(input_ids, attention_mask, token_type_ids, position_ids, turn_ids, role_ids)

        q_self_output = self_output * a_attention_mask.unsqueeze(-1)
        r_self_output = self_output * b_attention_mask.unsqueeze(-1)

        self_output = self_output * attention_mask.unsqueeze(-1)
        w = torch.matmul(q_self_output, r_self_output.transpose(-1, -2))

        if turn_ids is not None:
            view_turn_mask = turn_ids.unsqueeze(1).repeat(1, config.max_position_embeddings, 1)
            view_turn_mask_transpose = view_turn_mask.transpose(2, 1)
            view_range_mask = torch.where(abs(view_turn_mask_transpose - view_turn_mask) <= 1000,
                                          torch.ones_like(view_turn_mask),
                                          torch.zeros_like(view_turn_mask))
            filtered_w = w * view_range_mask

        q_cross_output = torch.matmul(filtered_w.permute(0, 2, 1), q_self_output)
        r_cross_output = torch.matmul(filtered_w, r_self_output)

        q_self_output = self.avg(q_self_output, a_attention_mask)
        q_cross_output = self.avg(q_cross_output, b_attention_mask)
        r_self_output = self.avg(r_self_output, b_attention_mask)
        r_cross_output = self.avg(r_cross_output, a_attention_mask)

        self_output = self.avg(self_output, attention_mask)
        q_self_output = q_self_output.view(-1, self.sample_nums, self.config.hidden_size)
        q_cross_output = q_cross_output.view(-1, self.sample_nums, self.config.hidden_size)
        r_self_output = r_self_output.view(-1, self.sample_nums, self.config.hidden_size)
        r_cross_output = r_cross_output.view(-1, self.sample_nums, self.config.hidden_size)

        self_output = self_output.view(-1, self.sample_nums, self.config.hidden_size)
        pooled_output = pooled_output.view(-1, self.sample_nums, self.config.hidden_size)

        output = self_output[:, 0, :]
        q_output = q_self_output[:, 0, :]
        r_output = r_self_output[:, 0, :]
        q_contrastive_output = q_cross_output[:, 0, :]
        r_contrastive_output = r_cross_output[:, 0, :]

        logit_q = []
        logit_r = []
        for i in range(self.sample_nums):
            cos_q = self.calc_cos(q_self_output[:, i, :], q_cross_output[:, i, :])
            cos_r = self.calc_cos(r_self_output[:, i, :], r_cross_output[:, i, :])
            logit_r.append(cos_r)
            logit_q.append(cos_q)

        logit_r = torch.stack(logit_r, dim=1)
        logit_q = torch.stack(logit_q, dim=1)

        loss_r = self.calc_loss(logit_r, labels)
        loss_q = self.calc_loss(logit_q, labels)

        if strategy not in ['mean', 'mean_by_role']:
            raise ValueError('Unknown strategy: [%s]' % strategy)

        output_dict = {'loss': loss_r + loss_q,
                       'final_feature': output if strategy == 'mean' else q_output + r_output,
                       'q_feature': q_output,
                       'r_feature': r_output,
                       'attention': w}

        return output_dict

    def encoder(self, *x):
        input_ids, attention_mask, token_type_ids, position_ids, turn_ids, role_ids = x    

        output = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            output_hidden_states=True,
                            return_dict=True)
        all_output = output['hidden_states']
        pooler_output = output['pooler_output']
        return all_output[-1], pooler_output

    def calc_cos(self, x, y):
        cos = torch.cosine_similarity(x, y, dim=1)
        cos = cos / 1.0 # cos = cos / 2.0
        return cos

    def calc_loss(self, pred, labels):
        loss = -torch.mean(self.log_softmax(pred) * labels)
        return loss

    def get_result(self):
        return self.result

    def get_labels_data(self):
        return self.labels_data

In [5]:
class BertFeatures():
    def __init__(self, input_ids, input_mask, segment_ids, role_ids, label_id, turn_ids=None, position_ids=None, guid=None):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.role_ids = role_ids
        self.turn_ids = turn_ids
        self.position_ids = position_ids
        self.label_id = label_id
        self.guid = guid

        self.batch_size = len(self.input_ids)
import codecs  



# "datasets/doc2dial/train.tsv"

def get_features(file_path : str):
    features = []
    with codecs.open(file_path, "r", "utf") as f: 
        examples = [] 
        for line in f: 
            line = [s.strip() for s in line.split('\t') if s.strip()]
            role, session, label = line[0], line[1], line[2]

            examples.append((role, session, label))

        for example in examples: 
            samples = example[1].split("|")
            roles = [int(r) for r in example[0].split("|")] \
                    if example[0].find("#") != -1 \
                    else [int(r) for r in example[0]]
        
        
            sample_input_ids = []
            sample_segment_ids = []
            sample_role_ids = []
            sample_input_mask = []
            sample_turn_ids = []
            sample_position_ids = []

            for t, s in enumerate(samples):
                text_tokens = []
                text_turn_ids = []
                text_role_ids = []

                texts = s.split("#")

                # bert-token:     [cls]  token   [sep]  token
                # roberta-token:   <s>   token   </s>   </s> token
                text_tokens.append(start_token)
                text_turn_ids.append(0)
                text_role_ids.append(roles[0])

                for i, text in enumerate(texts): 

                    tokenized = tokenizer.tokenize(text)
                    text_tokens.extend(tokenized)
                    text_turn_ids.extend([i] * len(tokenized))
                    text_role_ids.extend([roles[i]] * len(tokenized))

                    if i != (len(text) - 1): 
                        text_tokens.append(sep_token)
                        text_turn_ids.append(i)
                        text_role_ids.append(roles[i])

                max_seq_length = config.max_position_embeddings

                text_tokens = text_tokens[:max_seq_length]
                text_turn_ids = text_turn_ids[:max_seq_length]
                text_role_ids = text_role_ids[:max_seq_length]

                text_input_ids = tokenizer.convert_tokens_to_ids(text_tokens)


                text_input_ids += [pad_token_id] * (max_seq_length - len(text_tokens))
                text_input_mask = [1] * len(text_tokens) + [0] * (max_seq_length - len(text_tokens))
                text_segment_ids = [0] * max_seq_length
                text_position_ids = list(range(len(text_tokens))) + [0] * (max_seq_length - len(text_tokens))
                text_turn_ids += [0] * (max_seq_length - len(text_tokens))
                text_role_ids += [0] * (max_seq_length - len(text_tokens))


                assert len(text_input_ids) == max_seq_length
                assert len(text_input_mask) == max_seq_length
                assert len(text_segment_ids) == max_seq_length
                assert len(text_position_ids) == max_seq_length
                assert len(text_turn_ids) == max_seq_length
                assert len(text_role_ids) == max_seq_length

                sample_input_ids.append(text_input_ids)
                sample_turn_ids.append(text_turn_ids)
                sample_role_ids.append(text_role_ids)
                sample_segment_ids.append(text_segment_ids)
                sample_position_ids.append(text_position_ids)
                sample_input_mask.append(text_input_mask) 


            n_neg = 9
            label_id = [1] + [0] * n_neg
            bert_feature = BertFeatures(input_ids=sample_input_ids,
                                        input_mask=sample_input_mask,
                                        segment_ids=sample_segment_ids,
                                        role_ids=sample_role_ids,
                                        turn_ids=sample_turn_ids,
                                        position_ids=sample_position_ids,
                                        label_id=label_id,
                                        guid=[None] * (1 + n_neg))

            features.append(bert_feature)
    return features

features = get_features("datasets/doc2dial/train.tsv")
clustering_test = get_features("datasets/doc2dial/clustering_test.tsv")

In [6]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, DistributedSampler
import torch 
all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
all_role_ids = torch.tensor([f.role_ids for f in features], dtype=torch.long)
all_turn_ids = torch.tensor([f.turn_ids for f in features], dtype=torch.long)
all_position_ids = torch.tensor([f.position_ids for f in features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)

train_data = TensorDataset(all_input_ids,
                            all_input_mask,
                            all_segment_ids,
                            all_role_ids,
                            all_turn_ids,
                            all_position_ids,
                            all_label_ids)

train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data,
                        sampler=train_sampler,
                        batch_size=3)


clustering_all_input_ids = torch.tensor([f.input_ids for f in clustering_test], dtype=torch.long)
clustering_all_input_mask = torch.tensor([f.input_mask for f in clustering_test], dtype=torch.long)
clustering_all_segment_ids = torch.tensor([f.segment_ids for f in clustering_test], dtype=torch.long)
clustering_all_role_ids = torch.tensor([f.role_ids for f in clustering_test], dtype=torch.long)
clustering_all_turn_ids = torch.tensor([f.turn_ids for f in clustering_test], dtype=torch.long)
clustering_all_position_ids = torch.tensor([f.position_ids for f in clustering_test], dtype=torch.long)
clustering_all_label_ids = torch.tensor([f.label_id for f in clustering_test], dtype=torch.long)

test_data = TensorDataset(clustering_all_input_ids,
                            clustering_all_input_mask,
                            clustering_all_segment_ids,
                            clustering_all_role_ids,
                            clustering_all_turn_ids,
                            clustering_all_position_ids,
                            clustering_all_label_ids)

torch.save(train_data, "training_cache.pt")
torch.save(test_data, "testing_cache.pt")

test_sampler = SequentialSampler(test_data)
test_loader = DataLoader(test_data, sampler = test_sampler, batch_size = 3)

## New Session here 

In [7]:

def feature_based_evaluation_at_once(features, labels, gpu_features=None, n_average=1, tsne_visualization_output=None, tasks=None, dtype='float64', logger=None, note=''):
    """
    Evaluate all metrics with features
    :param features:                        numpy.array
    :param labels:                          list
    :param n_average:
    :param tsne_visualization_output:
    :param tasks:
    :param dtype:
    :param logger:
    :param note:
    :return:
    """
    labels = np.array(labels).astype(int)
    if gpu_features is not None:
        gpu_labels = torch.tensor(labels, device=gpu_features.device)
    features = np.array(features).astype(dtype) if features is not None else None

    # n_classes
    label_set = set()
    for s in labels:
        label_set.add(s)

    # initialize
    RI, NMI, acc, purity = 0., 0., 0., 0.
    clustering_time, RI_time, NMI_time, acc_time, purity_time = 0., 0., 0., 0., 0.
    SR, SR_time = 0., 0.
    MRR, MAP, mrr_time, map_time, ranking_time, scoring_time = 0., 0., 0., 0., 0., 0.
    alignment, adjusted_alignment, uniformity = 0., 0., 0.
    align_uniform_time = 0.

    # KMeans
    if 'clustering' in tasks:
        # logger.info('KMeans Evaluation for %s tries.' % n_average)
        for _ in range(n_average):
            # clustering
            pre = time()
            clf = KMeans(n_clusters=len(label_set), max_iter=500, tol=1e-5)
            clf.fit(features)
            y_pred = clf.predict(features)
            clustering_time += (time() - pre) / n_average

            ## RI
            pre = time()
            RI += adjusted_rand_score(labels, y_pred) / n_average
            RI_time += (time() - pre) / n_average

            ## NMI
            pre = time()
            NMI += normalized_mutual_info_score(labels, y_pred) / n_average
            NMI_time += (time() - pre) / n_average

            ## acc
            pre = time()
            acc += get_accuracy(labels, y_pred) / n_average
            acc_time += (time() - pre) / n_average

            ## purity
            pre = time()
            purity += purity_score(labels, y_pred) / n_average
            purity_time += (time() - pre) / n_average

    if logger is not None:
        logger.info("\nclustering_task [%s]: RI: %s NMI: %s Acc: %s Purity: %s" % (note, RI, NMI, acc, purity))
        logger.info("\nSemantic Relatedness [%s]: SR: %s" % (note, SR))
        logger.info("\nSession Retrieval [%s]: MRR: %s MAP: %s" % (note, MRR, MAP))
        logger.info("\nRepresentation_Evaluation [%s]: Alignment: %.6f Alignment (adjusted): %.6f Uniformity: %.6f" % (note, alignment, adjusted_alignment, uniformity))

        tb = PrettyTable()
        tb.field_names = ['', 'RI', 'NMI', 'Acc', 'Purity', 'SR', 'MRR', 'MAP', 'Alignment', 'Adjusted Alignment', 'Uniformity']
        tb.add_row(['Metrics'] + ['%.2f' % (v * 100) for v in [RI, NMI, acc, purity, SR, MRR, MAP]] + ['%.2f' % v for v in [alignment, adjusted_alignment, uniformity]])
        tb.add_row(['Times'] + ['%.2f s' % v for v in [clustering_time/4 + RI_time,
                                                       clustering_time/4 + NMI_time,
                                                       clustering_time/4 + acc_time,
                                                       clustering_time/4 + purity_time,
                                                       scoring_time/3 + SR_time,
                                                       scoring_time/3 + ranking_time/2 + mrr_time,
                                                       scoring_time/3 + ranking_time/2 + map_time,
                                                       align_uniform_time/3,
                                                       align_uniform_time/3,
                                                       align_uniform_time/3]])
        logger.info('\n' + tb.__str__())

    return EvaluationResult(
        RI=RI,
        NMI=NMI,
        acc=acc,
        purity=purity,
        SR=SR,
        MRR=MRR,
        MAP=MAP,
        alignment=alignment,
        adjusted_alignment=adjusted_alignment,
        uniformity=uniformity
    )


In [8]:

def eval(model: DialogueTransformer):
    model.eval()
    features = [] 
    with torch.no_grad(): 
        for step, batch in enumerate(test_loader):
            batch = tuple(t.to(torch.device("mps")) for t in batch)
            output_dict = model(batch)

            role2feat = {'all': 'final_feature', 'p1': 'q_feature', 'p2': 'r_feature'}
            feature = output_dict[role2feat['all']]
            features.append(feature)
        features = torch.cat(features)


    test_path = os.path.join("datasets/doc2dial/clustering_test.tsv")
    with codecs.open(test_path, "r", "utf-8") as f:
        labels = [int(line.strip('\n').split("\t")[-1]) for line in f]
    evaluation_result = EvaluationResult()

    n_average = max(3, 10 - features.shape[0] // 500)
    er = feature_based_evaluation_at_once(features=features.cpu(),
                                            labels=labels,
                                            n_average=n_average,
                                            tsne_visualization_output=None,
                                            tasks=['clustering'],
                                            dtype='float32',
                                            logger=None,
                                            note=','.join(['test', 'mean', 'all']))
    evaluation_result.RI = er.RI
    evaluation_result.NMI = er.NMI
    evaluation_result.acc = er.acc
    evaluation_result.purity = er.purity

    return evaluation_result 

In [9]:
from optimization import BERTAdam


d2vmodel = DialogueTransformer().to(torch.device("mps"))

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if n not in no_decay], 'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if n in no_decay], 'weight_decay_rate': 0.0}]
optimizer = BERTAdam(optimizer_grouped_parameters, lr=1e-5, warmup=0.1, t_total=line_statistics("datasets/doc2dial/train.tsv"))

res = EvaluationResult()

In [10]:
d2vmodel.train()
for step, batch in enumerate(train_loader):
    batch = tuple(t.to(torch.device("mps")) for t in batch)
    output_dict = d2vmodel(batch, strategy='mean_by_role')
    break

In [11]:
steps = 0 
d2vmodel.train()
for epoch in range(10):
    for step, batch in enumerate(train_loader):
        batch = tuple(t.to(torch.device("mps")) for t in batch)
        output_dict = d2vmodel(batch, strategy='mean_by_role')
        loss = output_dict['loss']

        loss = loss / 1.0
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        if steps % 20 == 0: 
            print("Loss: ", loss.item())
        if steps % 100 == 0:
            new_res = eval(d2vmodel)
            print(new_res.acc, res.acc)
            if new_res > res:
                print("new best: ", new_res.acc)
                torch.save(d2vmodel.state_dict(), "dial2vec_model.bin")
                res = new_res


            d2vmodel.train()

        steps += 1 

RuntimeError: MPS backend out of memory (MPS allocated: 60.29 GB, other allocations: 684.78 MB, max allowed: 61.20 GB). Tried to allocate 604.70 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).